# Lab01 - Busca Booleana

## Indice invertido

O método a seguir permite construir um indice invertido.

In [124]:
import nltk
import pandas as p
import numpy as np
from scipy import sparse
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from collections import OrderedDict

inverted_index = {}
def build_inverted_index(data):
    for index, doc in data.iteritems():
        doc = doc.lower()
        words = [str(word) for word in word_tokenize(doc)]
        for word in words:
            inverted_index.setdefault(word, []).append(index + 1)

    return inverted_index

## Implementação dos métodos de busca

Criando um método de busca genérico podendo receber pesquisas com apenas um termo, dois termos com *and* ou com *or*.

In [9]:
def search(term):
    term = n.word_tokenize(term)

    if len(term) == 1:
        return _search_one_term(term[0])
    elif len(term) == 3 and term[1].upper() == "AND":
        return _search_and(term[0], term[2])
    elif len(term) == 3 and term[1].upper() == "OR":
        return _search_or(term[0], term[2])

Método de busca com apenas um termo.

In [10]:
def _search_one_term(term):
    if inverted_index.has_key(term.lower()):
        return sorted(list(set(inverted_index[term.lower()])))
    else:
        return None

Metodo de busca *and* com apenas dois termos.

In [11]:
def _search_and(first_term, second_term):
    if inverted_index.has_key(first_term.lower()) and inverted_index.has_key(second_term.lower()):
        return sorted(list(set(inverted_index[first_term.lower()]) & set(inverted_index[second_term.lower()])))
    else:
        return None

Metodo de busca *or* com apenas dois termos.

In [12]:
def _search_or(first_term, second_term):
    if inverted_index.has_key(first_term.lower()) and inverted_index.has_key(second_term.lower()):
        return sorted(list(set(inverted_index[first_term.lower()]) | set(inverted_index[second_term.lower()])))
    elif inverted_index.has_key(first_term.lower()):
        return sorted(list(set(inverted_index[first_term.lower()])))
    elif inverted_index.has_key(second_term.lower()):
        return sorted(list(set(inverted_index[second_term.lower()])))
    else:
        return None

## Matriz

In [112]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

## Importação do data-set

In [116]:
data = p.read_csv("data-set/estadao_noticias_eleicao.csv", encoding = "utf-8")
data = data.replace(np.NAN, "")
content = data.titulo + " " + data.subTitulo + " " + data.conteudo


### Removendo pontuação

In [118]:
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = content.apply(lambda text: tokenizer.tokenize(text.lower()))

### Removendo stopwords

In [119]:
stopword_ = stopwords.words('portuguese')
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

### Transformando lista de listas em uma lista

In [120]:
tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

In [126]:
matrix, vocab = co_occurrence_matrix(tokens)
consultable_matrix = matrix.tocsr()
inverted_vocab = {vocab[key]: key for key in vocab}

## 2

In [125]:
def get_co_ocurrences(co_matrix, inv_vocab, term, N=3):
    np_array = np.reshape(co_matrix[term].toarray(), -1)
    return list(OrderedDict({inv_vocab[idx]: np_array[idx] for idx in (-np_array).argsort()[:N]}).keys())

## Chamada ao método de contrução de indice invertido


In [60]:
build_inverted_index(content)
print (inverted_index)

{'pt': [1, 1, 1, 1, 1, 3, 3, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8], 'espera': [1, 7], '30': [1, 1, 1, 7, 7], 'mil': [1, 1, 1, 1, 7, 7, 7, 7, 7, 7, 7, 7], 'pessoas': [1, 1, 1, 1, 1, 1, 1, 1, 7, 7, 7, 8, 8, 8, 10], 'em': [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 10, 10, 10, 10, 10, 10], 'festa': [1, 1, 1, 1], 'na': [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8], 'esplanada': [1, 1, 1], 'objetivo': [1, 1, 7, 7,

## Sanity Check

In [15]:
assert len(search("Campina AND Grande")) == 12

In [16]:
assert len(search("Campina OR Grande")) == 1656

In [17]:
assert search("Campina AND Grande") == [1068, 1370, 1770, 1952, 1987, 2763, 2777, 2779, 4802, 5382, 5870, 6694]

Para validar um busca *or* podemos usar a seguinte propriedade de operação de conjuntos

![OR](https://njadnrfof9.execute-api.us-east-1.amazonaws.com/prod/svg?tex=%7CA%5C%3A%5Ccup%5C%3AB%7C%5C%3A%3D%5C%3A%7CA%7C%5C%3A%2B%5C%3A%7CB%7C%5C%3A%5C%3A-%5C%3A%7CA%5Ccap%20B%7C)

In [18]:
assert len(search("Campina OR Grande")) == len(search("Campina")) + len(search("Grande")) - len(search("Campina AND Grande"))

## Exemplos de execução

debate *AND* presidencial

In [19]:
assert len(search("debate AND presidencial")) == 201

debate *OR* presidencial

In [20]:
assert len(search("debate OR presidencial")) == 1770

presidenciáveis *AND* corruptos

In [21]:
assert len(search("presidenciáveis AND corruptos")) == 0

presidenciáveis *OR* corruptos

In [22]:
assert len(search("presidenciáveis OR corruptos")) == 164

Belo *OR* Horizonte

In [23]:
assert len(search("Belo OR Horizonte")) == 331

Belo *AND* Horizonte

In [24]:
assert len(search("Belo AND Horizonte")) == 242